# SVHN

## Setup
### Preparation for Google Colab

In [1]:
google_colab = False

In [2]:
# install
try:
    import binarybrain as bb
except:
    if google_colab:
        !pip install pybind11
        %cd /content
        !nvcc -V
        !sudo rm -fr BinaryBrain
        !rm -fr BinaryBrain
        !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
        %cd /content/BinaryBrain/python
        !sh copy_src.sh
        !python3 setup.py build
        !python3 setup.py develop

        import binarybrain as bb

In [3]:
# mount
if google_colab:
    from google.colab import drive 
    drive.mount('/content/drive')
    %cd /content/drive/My Drive

### setup GPU

In [4]:
# select CUDA device
device = 0
bb.set_device(device)
prop = bb.get_device_properties(device)
print(prop)

# setup OpenMP
bb.omp_set_num_threads(4)


name                     : GeForce GTX 1660 SUPER
totalGlobalMem           : 6442450944
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1800000
totalConstMem            : 65536
major                    : 7
minor                    : 5
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 22
kernelExecTimeoutEnabled : 1
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




## Main code

In [5]:
import binarybrain as bb
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import os
import sys
import scipy.io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import OrderedDict

In [6]:
data_path    = 'SvhnCnnLutNet'
network_name = 'SvhnCnnLutNet'

mini_batch_size   = 32
binary_modulation = 1
with_fine_tuning  = True

In [7]:
def plot_image(img):
    img = img.reshape(3, 32, 32).transpose(1, 2, 0)
    plt.imshow(img)

## Train
### setup tain data

In [8]:
train_data = sio.loadmat('train_32x32.mat')
test_data  = sio.loadmat('test_32x32.mat')

train_len = len(train_data['y'])
test_len = len(test_data['y'])

td = {}
td['x_train'] = train_data['X'].transpose(3, 2, 0, 1).reshape(-1, 3*32*32).astype(np.float32) / 255.0
td['x_test']  = test_data['X'].transpose(3, 2, 0, 1).reshape(-1, 3*32*32).astype(np.float32) / 255.0
td['t_train'] = np.zeros((train_len, 10), dtype=np.float32)
td['t_test']  = np.zeros((test_len, 10), dtype=np.float32)
for i in range(train_len):
    td['t_train'][i][train_data['y'][i][0] % 10] = 1.0
for i in range(test_len):
    td['t_test'][i][test_data['y'][i][0] % 10] = 1.0
td['x_shape'] = [32, 32, 3]
td['t_shape'] = [10]

## Buld network

### define classes for distillation

In [9]:
def create_conv_layer(sub_layers, w, h, padding='valid'):
    sub_net = bb.Sequential.create()
    for layer in sub_layers:
        sub_net.add(layer)
    return bb.LoweringConvolutionBit.create(sub_net, w, h, 1, 1, padding=padding)

In [10]:
net = bb.Sequential.create()

# input
net.add(bb.BitEncodeBit.create(6, [32, 32, 3*6]))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 36*6], batch_norm=False, connection='serial')], 1, 1))

# layer0 conv(32x32 -> 30x30)
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 36*6], batch_norm=False, connection='random')], 1, 1))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 36*6], batch_norm=True, connection='depthwise')], 3, 3))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 36],   batch_norm=True, connection='serial')], 1, 1))

# layer1 conv(30x30 -> 28x28)
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 36*6], batch_norm=True, connection='random')], 1, 1))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 36*6], batch_norm=True, connection='depthwise')], 3, 3))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 72],   batch_norm=True, connection='serial')], 1, 1))

# layer2  pooling(28x28 -> 14x14)
net.add(bb.MaxPoolingBit.create(2, 2))

# layer3 conv(14x14 -> 12x12)
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 72*6], batch_norm=False, connection='random')], 1, 1))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 72*6], batch_norm=True, connection='depthwise')], 3, 3))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 72],   batch_norm=True, connection='serial')], 1, 1))

# layer4 conv(12x12 -> 10x10)
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 72*6], batch_norm=True, connection='random')], 1, 1))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 72*6], batch_norm=True, connection='depthwise')], 3, 3))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 144],   batch_norm=True, connection='serial')], 1, 1))

# layer5 pooling (10x10 -> 5x5)
net.add(bb.MaxPoolingBit.create(2, 2))

# layer6 conv(5x5 -> 3x3)
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 144*6], batch_norm=False, connection='random')], 1, 1))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 144*6], batch_norm=True, connection='depthwise')], 3, 3))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 144],   batch_norm=True, connection='serial')], 1, 1))

# layer7 conv(3x3 -> 1x1)
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 144*6], batch_norm=True, connection='random')], 1, 1))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 144*6], batch_norm=True, connection='depthwise')], 3, 3))
net.add(create_conv_layer([bb.SparseLut6Bit.create([1, 1, 288],   batch_norm=True, connection='serial')], 1, 1))

# layer8
net.add(bb.SparseLut6Bit.create([3456], batch_norm=True,  connection='random'))
net.add(bb.SparseLut6Bit.create([ 576], batch_norm=True,  connection='serial'))

# layer9
net.add(bb.SparseLut6Bit.create([2160], batch_norm=True,  connection='random'))
net.add(bb.SparseLut6Bit.create([ 360], batch_norm=True,  connection='serial'))
net.add(bb.SparseLut6Bit.create([  60], batch_norm=False, connection='serial'))
net.add(bb.SparseLut6Bit.create([  10], batch_norm=False, connection='serial'))

# output
net.add(bb.BinaryToRealBit.create())

# set network
net.set_input_shape(td['x_shape'])
net.send_command('binary true')

In [11]:
print(net.get_info())

----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [BitEncode] 
   input  shape : {32, 32, 3}   output shape : {32, 32, 18}
   bit_size : 6
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (1, 1)
   input  shape : {32, 32, 18}   output shape : {32, 32, 216}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {32, 32, 18}     output shape : {1, 1, 18}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [SparseLut6] 
       input  shape : {1, 1, 18}       output shape : {1, 1, 216}
       binary : 1       batch_norm : 0
    ------------------------------------------------------------------
    [ConvolutionCol2Im] 
     input  shape : {1, 1, 216}

In [12]:
loss    = bb.LossSoftmaxCrossEntropy.create()
metrics = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()

optimizer.set_variables(net.get_parameters(), net.get_gradients())
runner = bb.Runner(net, network_name, loss, metrics, optimizer)

log = runner.fitting(td, epoch_size=10, mini_batch_size=mini_batch_size, file_write=False, file_read=True)

[load] SvhnCnnLutNet_net.json


KeyboardInterrupt: 